In [15]:
import pandas as pd

apld_news_sentiment = pd.read_csv('../data/unprocessed data/APLD_news_sentiment.csv')
cney_news_sentiment = pd.read_csv('../data/unprocessed data/CNEY_news_sentiment.csv')
combined_stock_data = pd.read_csv('../data/unprocessed data/combined_stock_data.csv')
ktta_news_sentiment = pd.read_csv('../data/unprocessed data/KTTA_news_sentiment.csv')
onco_news_sentiment = pd.read_csv('../data/unprocessed data/ONCO_news_sentiment.csv')
tnxp_news_sentiment = pd.read_csv('../data/unprocessed data/TNXP_news_sentiment.csv')

In [18]:
apld_news_sentiment['Date Published'] = pd.to_datetime(apld_news_sentiment['Date Published'], format='%Y%m%dT%H%M%S')
cney_news_sentiment['Date Published'] = pd.to_datetime(cney_news_sentiment['Date Published'], format='%Y%m%dT%H%M%S')
ktta_news_sentiment['Date Published'] = pd.to_datetime(ktta_news_sentiment['Date Published'], format='%Y%m%dT%H%M%S')
onco_news_sentiment['Date Published'] = pd.to_datetime(onco_news_sentiment['Date Published'], format='%Y%m%dT%H%M%S')
tnxp_news_sentiment['Date Published'] = pd.to_datetime(tnxp_news_sentiment['Date Published'], format='%Y%m%dT%H%M%S')

# Convert 'Date' column in the combined stock data to datetime and remove timezone info
combined_stock_data['Date'] = pd.to_datetime(combined_stock_data['Date'], utc=True).dt.tz_convert(None)

def preprocess_and_fill(stock_data, sentiment_data, stock_name):
    stock_filtered = stock_data[['Date', f'{stock_name}_Close', f'{stock_name}_Volume']].copy()
    
    # Merge the sentiment data with the filtered stock data
    merged_data = pd.merge_asof(stock_filtered.sort_values('Date'),
                                sentiment_data.sort_values('Date Published'),
                                left_on='Date', right_on='Date Published',
                                direction='backward')
    
    # Sort by date in descending order and forward fill missing sentiment values
    merged_data = merged_data.sort_values(by='Date', ascending=False)
    merged_data['Sentiment'] = merged_data['Sentiment'].ffill().bfill().fillna(0)  # Forward, backward fill and neutral sentiment imputation
    
    # Apply a rolling average (7-day window) to smooth out sentiment data
    merged_data['Sentiment'] = merged_data['Sentiment'].rolling(window=7, min_periods=1).mean()
    
    return merged_data

# Apply the function to datasets
apld_merged = preprocess_and_fill(combined_stock_data, apld_news_sentiment, 'APLD')
cney_merged = preprocess_and_fill(combined_stock_data, cney_news_sentiment, 'CNEY')
ktta_merged = preprocess_and_fill(combined_stock_data, ktta_news_sentiment, 'KTTA')
onco_merged = preprocess_and_fill(combined_stock_data, onco_news_sentiment, 'ONCO')
tnxp_merged = preprocess_and_fill(combined_stock_data, tnxp_news_sentiment, 'TNXP')

# Save processed data
apld_merged.to_csv('../data/processed data/APLD_merged.csv', index=False)
cney_merged.to_csv('../data/processed data/CNEY_merged.csv', index=False)
ktta_merged.to_csv('../data/processed data/KTTA_merged.csv', index=False)
onco_merged.to_csv('../data/processed data/ONCO_merged.csv', index=False)
tnxp_merged.to_csv('../data/processed data/TNXP_merged.csv', index=False)

                   Date  APLD_Close  APLD_Volume  \
247 2024-09-25 04:00:00        7.67     19647800   
246 2024-09-24 04:00:00        7.08     15410200   
245 2024-09-23 04:00:00        6.70     15381500   
244 2024-09-20 04:00:00        6.02     17700800   
243 2024-09-19 04:00:00        5.56     16552600   

                                                 Title  \
247  CORZ, WULF and APLD Stocks Set to Boom? Thank ...   
246  Applied Digital Sets Fiscal First Quarter 2025...   
245  Applied Announces New Innovation Across Its Di...   
244  Applied Announces New Innovation Across Its Di...   
243  Applied Announces New Innovation Across Its Di...   

                                               Summary  Sentiment  \
247  Data centers, AI, and Bitcoin mining are three...   0.000000   
246  DALLAS, Sept. 23, 2024 ( GLOBE NEWSWIRE ) -- A...   0.000000   
245  Chicago, IL., Sept. 11, 2024 ( GLOBE NEWSWIRE ...   0.083333   
244  Chicago, IL., Sept. 11, 2024 ( GLOBE NEWSWIRE ...   0.125